In [6]:
using Pkg; Pkg.activate("../."); Pkg.instantiate()
using Plots; gr()
#using SparseArrays
using Statistics
using ModelingToolkit
using LinearAlgebra
#using HDF5
using JLD
#using Infiltrator
using Zygote
using PaddedViews
using Flux
using Flux: @epochs
#using DiffEqOperators
using Tullio

  Activating environment at `~/Dropbox/Glacier UDE/ODINN_toy/scripts/Project.toml`
Precompiling project...
  ? DiffEqBase
  ? DiffEqJump
  ? ModelingToolkit
  ? DiffEqOperators


LoadError: ArgumentError: Package Tullio not found in current path:
- Run `import Pkg; Pkg.add("Tullio")` to install the Tullio package.


In [3]:
nx, ny = 100, 100 # Size of the grid
Δx, Δy = 1, 1
Δt = 0.01
t₁ = 1

D₀ = 1
tolnl = 1e-4
itMax = 100
damp = 0.85
dτsc   = 1.0/3.0
ϵ     = 1e-4            # small number
cfl  = max(Δx^2,Δy^2)/4.1;

### Dynamical functions

In [4]:
function heatflow(T, D::Real, p, tol=Inf)
   
    Δx, Δy, Δt, t₁ = p
    
    total_iter = 0
    t = 0
    
    while t < t₁
        
        iter = 1
        err = 2 * tolnl
        Hold = copy(T)
        dTdt = zeros(nx, ny)
        err = Inf 
        
        while iter < itMax+1 && tol <= err
            
            Err = copy(T)
                    
            F, dτ = Heat(T, D, p)
            
            @tullio ResT[i,j] := -(T[i,j] - Hold[i,j])/Δt + F[pad(i-1,1,1),pad(j-1,1,1)] 
            
            dTdt_ = copy(dTdt)
            @tullio dTdt[i,j] := dTdt_[i,j]*damp + ResT[i,j]
            
            T_ = copy(T)
            #@tullio T[i,j] := max(0.0, T_[i,j] + dTdt[i,j]*dτ[pad(i-1,1,1),pad(j-1,1,1)]) 
            @tullio T[i,j] := max(0.0, T_[i,j] + dτ * dTdt[i,j])
            
            Zygote.ignore() do
                Err .= Err .- T
                err = maximum(Err)
            end 
            
            iter += 1
            total_iter += 1
            
        end
        
        t += Δt
        
    end
    
    return T
    
end

LoadError: LoadError: UndefVarError: @tullio not defined
in expression starting at In[4]:22

In [5]:
function Heat(T, D, p)
   
    Δx, Δy, Δt, t₁ = p
    
    #dTdx = diff(S, dims=1) / Δx
    #dTdy = diff(S, dims=2) / Δy

    dTdx_edges = diff(T[:,2:end - 1], dims=1) / Δx
    dTdy_edges = diff(T[2:end - 1,:], dims=2) / Δy
    
    Fx = -D * dTdx_edges
    Fy = -D * dTdy_edges    
    F = .-(diff(Fx, dims=1) / Δx .+ diff(Fy, dims=2) / Δy) 

    dτ = dτsc * min( 10.0 , 1.0/(1.0/Δt + 1.0/(cfl/(ϵ + D))))
    
    return F, dτ
 
end

Heat (generic function with 1 method)

### Initial simulation

In [ ]:
p = (Δx, Δy, Δt, t₁)

T₀ = [ 250 * exp( - ( (i - nx/2)^2 + (j - ny/2)^2 ) / 300 ) for i in 1:nx, j in 1:ny ]

T₁ = copy(T₀)
T₁ = heatflow(T₁, D₀, p, 1e-1)

In [ ]:
heatmap(T₀, clim=(0, maximum(T₀)))

In [ ]:
heatmap(T₁, clim=(0, maximum(T₀)))

Notice that the changes in the temperature field are really small:

In [ ]:
sqrt( sum((T₁.-T₀).^2) / (nx * ny) )